In [70]:
import os
import glob
import folium
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd
import time
from IPython.display import clear_output

In [71]:
file_paths = [
    "./data/combined_train_schedules_rer_A.csv/",
    "./data/combined_train_schedules_rer_B.csv/",
    "./data/combined_train_schedules_rer_C.csv/",
    "./data/combined_train_schedules_rer_D.csv/",
    "./data/combined_train_schedules_rer_E.csv/",
]

# Define color mapping for RER lines
rer_colors = {
    'A': 'red',
    'B': 'blue',
    'C': 'goldenrod',
    'D': 'green',
    'E': 'purple'
}

In [72]:
file_patterns = [os.path.join(path, "part-*.csv") for path in file_paths]

# Use glob to get all part file paths
all_files = [file for pattern in file_patterns for file in glob.glob(pattern)]

# Initialize an empty list to hold DataFrames
dataframes = []

# Iterate over the file paths and read each CSV file
for file in all_files:
    df = pd.read_csv(file)
    dataframes.append(df)

# Concatenate all DataFrames into a single DataFrame
train_data = pd.concat(dataframes, ignore_index=True)

# Modify the journey_ids list to include the line information
journey_ids = [
    ("A", "vehicle_journey:SNCF:2024-07-08:BORA13:1187:RapidTransit"),
    ("A", "vehicle_journey:SNCF:2024-07-08:NAJA22:1187:RapidTransit"),
    ("B", "vehicle_journey:SNCF:2024-07-08:EACE07:1187:RapidTransit"),
    ("C", "vehicle_journey:SNCF:2024-07-08:141186-141187:1187:RapidTransit"),
    ("D", "vehicle_journey:SNCF:2024-07-08:122302-122303:1187:RapidTransit"),
    ("E", "vehicle_journey:SNCF:2024-07-08:116192:1187:RapidTransit"),
]

journeys = {}

for line, journey_id in journey_ids:
    journey = train_data[train_data["journey_id"] == journey_id]
    geometry = [Point(xy) for xy in zip(journey["longitude"], journey["latitude"])]
    gdf_journey = gpd.GeoDataFrame(journey, geometry=geometry)

    # Ajouter le GeoDataFrame à la liste correspondante dans journeys
    if line in journeys:
        journeys[line].append(gdf_journey)
    else:
        journeys[line] = [gdf_journey]

In [73]:
def creer_carte_folium(journeys, current_indices):
    # Créer une carte centrée sur Paris
    carte = folium.Map(location=[48.8566, 2.3522], zoom_start=11)

    # Dictionnaire pour stocker les éléments de la légende
    legend_html = {}

    for line, gdf_journeys in journeys.items():
        for j, gdf_journey in enumerate(gdf_journeys):
            current_index = current_indices[line][j]
            color = rer_colors[line]

            # Créer une liste de coordonnées pour la ligne complète
            locations = [(lat, lon) for lon, lat in zip(gdf_journey.geometry.x, gdf_journey.geometry.y)]
            folium.PolyLine(locations, color=color, weight=2, opacity=0.5, dash_array="5, 5", popup=f"RER {line} Path").add_to(carte)

            # Créer une liste de coordonnées pour la progression actuelle
            current_locations = locations[:current_index + 1]

            # Vérifier si current_locations a au moins deux coordonnées
            if len(current_locations) >= 2:
                folium.PolyLine(current_locations, color=color, weight=4, popup=f"RER {line} Progress").add_to(carte)

                # Add a marker for the current train position
                current_position = current_locations[-1]
                folium.CircleMarker(current_position, radius=5, color=color, fill=True, fill_color=color, popup=f"RER {line}").add_to(carte)

            # Add markers for the stations
            for idx, row in gdf_journey.iterrows():
                station_label = f"{line}{idx + 1}"
                folium.CircleMarker(
                    [row.geometry.y, row.geometry.x],
                    radius=3,
                    color=color,
                    fill=True,
                    fill_color='white',
                    popup=f"{station_label}: {row['stop_point_name']}"
                ).add_to(carte)

        # Ajouter un élément à la légende pour la ligne actuelle
        legend_html[line] = f'<span style="background-color: {rer_colors[line]}; display: inline-block; width: 10px; height: 10px; margin-right: 5px;"></span>RER {line} '

    # Créer le HTML de la légende
    legend_html = '<div style="position: fixed; bottom: 10px; left: 50%; transform: translateX(-50%); background-color: white; padding: 5px; border-radius: 5px; z-index: 1000;">' + ''.join(legend_html.values()) + '</div>'

    # Ajouter la légende à la carte
    carte.get_root().html.add_child(folium.Element(legend_html))

    return carte

In [74]:
# Simulation de la progression du trajet
max_length = max(len(gdf_journey) for gdf_journeys in journeys.values() for gdf_journey in gdf_journeys)
current_indices = {line: [0] * len(gdf_journeys) for line, gdf_journeys in journeys.items()}

for i in range(max_length):
    for line, gdf_journeys in journeys.items():
        for j, gdf_journey in enumerate(gdf_journeys):
            if current_indices[line][j] < len(gdf_journey) - 1:
                current_indices[line][j] += 1
    clear_output(wait=True)
    carte = creer_carte_folium(journeys, current_indices)
    display(carte)
    time.sleep(1)